In [ ]:
#importing libraries
import pandas as pd
import sqlite3
import matplotlib
import seaborn

In [ ]:
#configuring mysql into jupyter
%load_ext sql
%config SqlMagic.autopandas=True

In [17]:
#converting the statement PDF file to a data frame
import tabula

#path to the PDF file
pdf_path = "/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.pdf"

# password to unlock the PDF file
password = "900365"

# options for the PDF extraction
options = {
    "pages": "all",        # Extracting data from all the pages
    "password": password   # password to unlock the PDF
}

# read tables from a PDF file and save them as CSV files
tabula.convert_into('/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.pdf', 'mpesa_statement.csv', output_format='csv', **options)

df = pd.read_csv('/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.csv', delimiter=';')

#dropping the unnamed column
#df = df.drop('Unnamed', axis=1)

# Print the extracted data
print(df)


Got stderr: Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true



                                                      ,
0                    TRANSACTION TYPE,PAID IN,PAID OUT,
1                        SEND MONEY:,0.00,"135,777.00",
2                    RECEIVED MONEY:,"156,781.66",0.00,
3                           AGENT DEPOSIT:,350.00,0.00,
4                    AGENT WITHDRAWAL:,0.00,"9,090.00",
...                                                 ...
7425  07******245 daniel njeri",Completed,"1,050.00"...
7426  Statement Verification Code,"To verify the val...
7427                       prompts to enter the code.",
7428                                                 ,,
7429                                         KZ3FXNRZ,,

[7430 rows x 1 columns]
